In [1]:
import subprocess, pathlib, os, time, optuna
import pandas as pd
from joblib import Parallel, delayed
from pymol import cmd
from IPython.display import Image
working_dir = pathlib.Path("/home/justin/InstaDeep/mutation_test")
os.chdir(working_dir)

/home/justin/miniconda3/envs/py311/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def fix_pdb(input_pdb_path:pathlib.Path):
    """Fix a PDB file 
        
    Correct missing residue number etc. to generate a correct PDB file.

    Parameters
    ----------
    input_pdb_path : pathlib.Path
        The path object of a query PDB file. 
    
    """
    cmd = f'/home/justin/InstaDeep/mutation_test/program/pdbfixer {input_pdb_path} --output {input_pdb_path.stem}_output.pdb'
    output_temp = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd=input_pdb_path.parent)
    output = str(output_temp.communicate())
    print (output)

In [3]:
def _find_current_dir(target_text:str, dir_or_file:str='file', current_path:pathlib.Path=pathlib.Path(os.getcwd())) -> list:
    """Find a desired directory or a file

    Private method used to find a desired directory or a file with a name of target texts recursively on a given path.

    Parameters
    ----------
    target_text : str
        A given text included in any file names.
    dir_or_file : str
        Specify a directory or a file to be found. By default 'file'.
    current_path : pathlib.Path   
        The path object indicating a start point for searching. By default current working directory.

    Returns
    -------
    list
        A list containing path objects of found files or directories. 

    """
    if dir_or_file == 'dir':
        return [item for item in current_path.rglob(target_text) if item.is_dir()]
    elif dir_or_file == 'file':
        return [item for item in current_path.rglob(target_text) if item.is_file()]
    else:
        print ("Please choose 'dir' or 'file'")

def _prepare_docking_setup(receptor_pdb:pathlib.Path, ligand_pdb:pathlib.Path, num_of_swarms:int=10, working_path:pathlib.Path=working_dir) -> tuple:
    """Generate a setup file for docking

    A private method generating a setup file used for docking.

    Parameters
    ----------
    receptor_pdb : pathlib.Path
                The path object of a receptor PDB file.
    ligand_pdb : pathlib.Path
                The path object of a ligand PDB file.
    num_of_swarms : int
                The number of swarms set a for docking event using lightdock. By default 10
    working_path : pathlib.Path
                The path object indicting where the docking being performed. By default current working directory. 
        
    Returns
    -------
    tuple
        A tuple containig a object of the working directory and swarm directories. 
    
    """
    output_temp = subprocess.Popen(['lightdock3_setup.py', receptor_pdb, ligand_pdb, '--noxt', '--noh', '--now', '-anm', '-s', str(num_of_swarms)], cwd=working_path, stdout = subprocess.PIPE)
    output = str(output_temp.communicate())
    print (output)
    docking_setup_file = _find_current_dir(target_text="setup.json", dir_or_file='file', current_path=working_path)
    swarm_dir = _find_current_dir(target_text="swarm_*", dir_or_file='dir')
    return docking_setup_file, swarm_dir

def _run_docking(docking_setup_file:pathlib.Path, swarm_id:int, simulation_step:int=100, cpu_num:int=1, working_path:pathlib.Path=working_dir):
    """Perform docking
    
    A private method used to run a docking using Lightdock.

    Parameters
    ----------
    docking_setup_file : pathlib.Path
        The path object of a docking setup file.
    swarm_id : int
        The number of swarms used for docking simulation.
    simulation_step : int 
        The number of steps used for docking simulation. By default 100 
    cpu_num : int
        The number of CPU cores used for docking simulation. By default 1
    working_path : pathlib.Path
        The path object indicting where the docking being performed. By default current working directory.

    """
    print (docking_setup_file, swarm_id)
    output_temp = subprocess.Popen(['lightdock3.py', docking_setup_file, str(simulation_step), '-l', str(swarm_id), '-c', str(cpu_num), '-s', 'sipper'], cwd=working_path, stdout = subprocess.PIPE)
    output = str(output_temp.communicate())
    print (output)

def perform_docking(receptor_pdb:pathlib.Path, ligand_pdb:pathlib.Path, simulation_step:int=100, cpu_num:int=1, num_of_swarms:int=10, working_path:pathlib.Path=working_dir):
    """Set up and perform a docking enent

    A interface that used to prepare and perform a docking event.

    Parameters
    ----------
    receptor_pdb : pathlib.Path
        The path object of a receptor PDB file.
    ligand_pdb : pathlib.Path
        The path object of a ligand PDB file.
    simulation_step : int
        The number of steps used for docking simulation. By default 100
    cpu_num : int 
        The number of CPU cores used for docking simulation. By default 1
    num_of_swarms : int 
        The number of swarms set a for docking event using lightdock. By default 10
    working_path : pathlib.Path
        The path object indicting where the docking being performed. By default current working directory.
        
    """
    try:
        docking_setup_file, swarm_dir = _prepare_docking_setup(receptor_pdb, ligand_pdb, num_of_swarms, working_path)
    except:
        print ("Error orccured in prepare_docking_setup")
    
    try:
        print ("setup", docking_setup_file)
        result = (Parallel(n_jobs=8)(delayed(_run_docking)(docking_setup_file[0], id, simulation_step, cpu_num, working_path) for id in range(num_of_swarms)))
    except:
        print ("Error orccured in run_docking")

In [4]:
def analysize_docking_result(receptor_pdb:pathlib.Path, ligand_pdb:pathlib.Path, step:int=100, generate_best_pdb:bool=False, pdb_num_for_generated:int=200, working_path:pathlib.Path=working_dir) -> tuple:
   """Analyse the result of a docking event

   Analyse docking event and generate a PDB file with the best score. 

   Parameters
   ----------
   receptor_pdb : pathlib.Path
      The path object of a receptor PDB file.
   ligand_pdb : pathlib.Path
      The path object of a ligand PDB file.
   step : int
      The number of simulation steps to be analysed. By default 100.
   generate_best_pdb : bool
      A bool variable to check whether a structure PDB file generated given a path. By default False.
   pdb_num_for_generated : int
      The number of PDB files to be generated. By default 200.
   working_path : pathlib.Path
      The path object indicting where PDB being generated. By default current working directory.
   
   Returns
   -------
   tuple
      A tuple containing the index and the best score of the structure. Or a path object of a PDB file of the best structure.
      
   """
   docking_result_path =  _find_current_dir(target_text=f"gso_{step}.*", dir_or_file='file', current_path=working_path)
   docking_result_path = sorted(docking_result_path, key=lambda i: int(str(i.parents[0]).split("_")[-1]))
   df = pd.DataFrame()
   for swarm, f in enumerate(docking_result_path):
      df_temp = pd.read_csv(f, skiprows=1, names=['Coordinates', 'RecID', 'LigID','Luciferin', 'Neighbors number', 'Vision Range', 'Scoring'])
      df_temp = pd.DataFrame([float(x.split(" ")[-1]) for x in df_temp["Scoring"].tolist()])
      df_temp["Swarm"] = f"swarm_{swarm}"
      df_temp["Path"] = f
      df = pd.concat([df, df_temp])
   df.rename({0:'Score'}, axis=1, inplace=True)
   df_swarm_group_mean = df.groupby('Swarm').mean()
   docking_result_dict = df_swarm_group_mean.sort_values(by="Score", ascending=False).head(1).to_dict()
   best_swarm = str(list(docking_result_dict["Score"].keys())[0]) 
   df_best = df.loc[df["Swarm"] == best_swarm]
   best_index = df_best['Score'].idxmax()
   best_score = df_best['Score'].loc[best_index]
   best_path = df_best['Path'].loc[best_index]
   df_swarm_group_mean.to_csv("./df_swarm_group_mean.csv")
   if generate_best_pdb:
      output_temp = subprocess.Popen(['lgd_generate_conformations.py', receptor_pdb, ligand_pdb, best_path, str(pdb_num_for_generated)], cwd=working_path, stdout = subprocess.PIPE)
      output = str(output_temp.communicate())
      print (output)
      return (best_index, best_score, pathlib.Path(best_path.parent, pathlib.Path(f"lightdock_{best_index}.pdb")))
   else:
      return (best_index, best_score)   

In [5]:
def identify_contributing_residues(ori_docking_structure_pdb_path:pathlib.Path, dist_cutoff:float=3.0) -> dict:
    """Identify contact residues contributing to docking

    Look for residues contributing to docking within a set distance cutoff.

    Parameters
    ----------
    ori_docking_structure_pdb_path : pathlib.Path
        The path object of a query PDB to be measured. 
    dist_cutoff : float
        The distance cutoff to define the contributing residues. By default 3.0

    Returns
    -------
    dict
        A dictionary containing the chain ID, ID, and amino acid code of contributing residues.
    
    """
    cmd.load(ori_docking_structure_pdb_path)
    cmd.select('contact_residue',f'chain E within {dist_cutoff} of chain A')
    my_space={'contact_residue_dict': {}}
    cmd.iterate('contact_residue', 'contact_residue_dict[resi]=(resn, chain)', space=my_space)
    return my_space

def mutate_protein(ori_docking_structure_pdb_path:pathlib.Path, origial_residue_dict:dict, mutated_residue_code:tuple, relax_steps:int=1000, working_path:pathlib.Path=working_dir):
    """Mutate residues of a query protein

    Mutate residues in a PDB structure and such structure is then optimised after mutation.

    Parameters
    ----------
    ori_docking_structure_pdb_path : pathlib.Path
        The path object of a PDB structure to be mutated. 
    origial_residue_dict : dict 
        A dictionary contains original residues to be replaced.
    mutated_residue_code : tuple
        A tuple contains the residues used for mutation.
    relax_steps : int
        A number of steps used to optimise a structure after mutation. By default 1000.
    working_path : pathlib.Path
        The path object used to save a resulting PDB file of mutated structure. By default current working directory.
    
    """
    aa_code_mapping_dict={
        'A': 'ALA', 'R': 'ARG', 'H': 'HIS', 'K': 'LYS', 'D': 'ASP', 'E': 'GLU',
        'S': 'SER', 'T': 'THR', 'N': 'ASN', 'Q': 'GLN', 'C': 'CYS', 'G': 'GLY',
        'P': 'PRO', 'V': 'VAL', 'I': 'ILE', 'L': 'LEU', 'M': 'MET', 'F': 'PHE',
        'Y': 'TYR', 'W': 'TRP',
    }
    
    cmd.delete('all')
    cmd.load(ori_docking_structure_pdb_path)
    print (str(ori_docking_structure_pdb_path.stem))

    # Turn on the mutagenesis wizard
    cmd.wizard("mutagenesis")
    cmd.do("refresh_wizard")

    print (f"Total {len(origial_residue_dict['contact_residue_dict'])} residues to be mutated")
    
    # mutate residues
    for index, (res_id, res_info) in enumerate(origial_residue_dict["contact_residue_dict"].items()):
        cmd.get_wizard().set_mode(aa_code_mapping_dict[mutated_residue_code[index]])
        cmd.get_wizard().do_select(f'{str(res_info[-1])}/{int(res_id)}/')
        print (f"{index+1}: Mutating from {str(res_info[-1])}/{int(res_id)}/ to {aa_code_mapping_dict[mutated_residue_code[index]]}")
    
        # Apply mutation
        cmd.frame(1)
        cmd.get_wizard().apply()

    # relax structure
    cmd.select('mutant', f'{str(res_info[-1])}/{int(res_id)}/')
    cmd.select('new', 'br. mutant gap 10')
    cmd.flag('fix', 'br. mutant gap 10')
    cmd.sculpt_activate(str(ori_docking_structure_pdb_path.stem))
    cmd.sculpt_iterate(str(ori_docking_structure_pdb_path.stem), cycles=relax_steps)
    cmd.sculpt_deactivate(str(ori_docking_structure_pdb_path.stem))
    cmd.save(f'{working_path}/mutation.pdb')
    cmd.delete('all')
    print ("Mutation done")

In [20]:
def objective(trial:optuna.trial.Trial, orig_score_AB:float, orig_score_AC:float, orig_A_pdb_path:pathlib.Path, orig_ligand_B_path:pathlib.Path, orig_ligand_C_path:pathlib.Path, contact_residue_dict:dict) -> float: 
    """Objective function used for residues (parameters) scanning 
    
    A method used to build and optimise a loss function in order to explore a better combination of residues.
    
    Parameters
    ----------
    trial : optuna.trial.Trial
        A optuna trial object performing parameter scan.
    orig_score_AB : float
        The docking score of the original structures between A and B.
    orig_score_AC : float
        The docking score of the original structures between A and C.
    orig_A_pdb_path : pathlib.Path
        The path object of the PDB file of protein A. 
    orig_ligand_B_path : pathlib.Path
        The path object of the PDB file of ligand B.
    orig_ligand_C_path : pathlib.Path
        The path object of the PDB file of ligand C.
    contact_residue_dict : dict
        The dictionary contains the contributing residues.
    
    Returns
    -------
    float
        A float number of a loss score.

    """
    mutpoint1 = trial.suggest_categorical('mutpoint1', ['C', 'L', 'K', 'D'])
    mutpoint2 = trial.suggest_categorical('mutpoint2', ['N', 'R', 'E', 'F'])
    mutpoint3 = trial.suggest_categorical('mutpoint3', ['N', 'D', 'H', 'L'])
    mutpoint4 = trial.suggest_categorical('mutpoint4', ['P', 'W', 'K', 'E'])
    mutpoint5 = trial.suggest_categorical('mutpoint5', ['R', 'K', 'D', 'I'])
    mutpoint6 = trial.suggest_categorical('mutpoint6', ['K', 'R', 'D', 'I'])
    mutpoint7 = trial.suggest_categorical('mutpoint7', ['S', 'I', 'D', 'K'])
    mutation_combination = (mutpoint1, mutpoint2, mutpoint3, mutpoint4, mutpoint5, mutpoint6, mutpoint7)
    
    print (f"Trial:{trial.number}: {mutation_combination}")
    trial_dir = pathlib.Path(working_dir, f'protein_design_trial/Trial_{trial.number}')
    docking_AB_dir = pathlib.Path(trial_dir, 'docking_AB')
    docking_AC_dir = pathlib.Path(trial_dir, 'docking_AC')
    
    subprocess.Popen(['mkdir', f'Trial_{trial.number}'], cwd=pathlib.Path(working_dir,'protein_design_trial'), stdout = subprocess.PIPE)
    time.sleep(3)

    subprocess.Popen(['mkdir', docking_AB_dir, docking_AC_dir], cwd=trial_dir, stdout = subprocess.PIPE)
    
    mutate_protein(orig_A_pdb_path, contact_residue_dict, mutation_combination, 1000, trial_dir)
    time.sleep(3)
    
    mutation_pdb_file_path_list = _find_current_dir('mutation.pdb', 'file', trial_dir)
    print (mutation_pdb_file_path_list)
    fix_pdb(mutation_pdb_file_path_list[0])
    time.sleep(3)
    
    mutation_pdb_file_path_list = _find_current_dir('mutation_output.pdb', 'file', trial_dir)
    print (mutation_pdb_file_path_list)
    subprocess.Popen(['cp', mutation_pdb_file_path_list[0], orig_ligand_B_path, docking_AB_dir], stdout = subprocess.PIPE)
    subprocess.Popen(['cp', mutation_pdb_file_path_list[0], orig_ligand_C_path, docking_AC_dir], stdout = subprocess.PIPE)
    time.sleep(3)
    
    mutation_pdb_file_path_list = _find_current_dir('mutation_output.pdb', 'file', docking_AB_dir)
    print (mutation_pdb_file_path_list[0], orig_ligand_B_path.name)
    perform_docking(mutation_pdb_file_path_list[0], pathlib.Path(docking_AB_dir, orig_ligand_B_path.name), simulation_step=50, cpu_num=1, num_of_swarms=10, working_path=docking_AB_dir) # Want lower    
    docking_result_tuple_AB = analysize_docking_result(mutation_pdb_file_path_list[0], pathlib.Path(docking_AB_dir, orig_ligand_B_path.name), 50, False, 200, docking_AB_dir)
    print (docking_result_tuple_AB)
    score_AB = float(docking_result_tuple_AB[-1])
    
    time.sleep(3)

    mutation_pdb_file_path_list = _find_current_dir('mutation_output.pdb', 'file', docking_AC_dir)
    perform_docking(mutation_pdb_file_path_list[0], pathlib.Path(docking_AC_dir, orig_ligand_C_path.name), simulation_step=50, cpu_num=1, num_of_swarms=10, working_path=docking_AC_dir) # Want equal or higher
    time.sleep(3)
    docking_result_tuple_AC = analysize_docking_result(mutation_pdb_file_path_list[0], pathlib.Path(docking_AC_dir, orig_ligand_C_path.name), 50, False, 200, docking_AC_dir)
    print (docking_result_tuple_AC)
    score_AC = float(docking_result_tuple_AC[-1])
    print (f"trail:{trial.number} score: ({score_AB}, {score_AC})")
    trial.set_user_attr('score_AB', score_AB)
    trial.set_user_attr('score_AC', score_AC)
    trial.set_user_attr('parameters', mutation_combination)

    score_AB-=orig_score_AB # more negative the better    
    score_AC-=orig_score_AC 
    loss = score_AB - score_AC*1000 if score_AC < 0 else score_AB

    trial.set_user_attr('loss', loss)

    return (loss)


### Protein preparation

In [ ]:
# Prepare the ACE2 structure from 6m0j.pdb and fix it using the fix_pdb funtion 
cmd.do('fetch 6m0j, type=pdb, async=0')
cmd.load(pathlib.Path(working_dir, '6m0j.pdb'))
cmd.remove('! chain A')
cmd.remove('! polymer')
cmd.save(pathlib.Path(working_dir, '6m0j_ACE2.pdb'))
cmd.delete('all')
fix_pdb(pathlib.Path(working_dir, '6m0j_ACE2.pdb'))

In [ ]:
# Prepare the Spike structure from 6m0j.pdb and fix it using the fix_pdb funtion 
cmd.load(pathlib.Path(working_dir,'6m0j.pdb'))
cmd.remove('! chain E')
cmd.remove('! polymer')
cmd.save(pathlib.Path(working_dir, '6m0j_Spike.pdb'))
cmd.delete('all')
fix_pdb(pathlib.Path(working_dir, '6m0j_Spike.pdb'))

In [ ]:
# Prepare the antibody structure from 7z0x.pdb and fix it using the fix_pdb funtion 
cmd.do('fetch 7z0x, type=pdb, async=0')
cmd.load(pathlib.Path(working_dir, '7z0x.pdb'))
cmd.remove('chain R')
cmd.remove('! polymer')
cmd.save(pathlib.Path(working_dir,'7z0x_antibody.pdb'))
cmd.delete('all')
fix_pdb(pathlib.Path(working_dir, '7z0x_antibody.pdb'))

### Dock originals

In [ ]:
# Perform docking bwtween the orginal protiens: Spike protein (Prot_A) and antibody (Prot_C)
perform_docking(pathlib.Path(working_dir, '6m0j_Spike_output.pdb'), pathlib.Path(working_dir, '6m0j_ACE2_output.pdb'), 50, 1, 10, working_dir)

(b'[lightdock3_setup] INFO: Ignoring OXT atoms\n[lightdock3_setup] INFO: Ignoring Hydrogen atoms\n[lightdock3_setup] INFO: Ignoring water\n[lightdock3_setup] INFO: Reading structure from /home/justin/InstaDeep/mutation_test/6m0j_Spike_output.pdb PDB file...\n[lightdock3_setup] INFO: 1536 atoms, 194 residues read.\n[lightdock3_setup] INFO: Ignoring OXT atoms\n[lightdock3_setup] INFO: Ignoring Hydrogen atoms\n[lightdock3_setup] INFO: Ignoring water\n[lightdock3_setup] INFO: Reading structure from /home/justin/InstaDeep/mutation_test/6m0j_ACE2_output.pdb PDB file...\n[lightdock3_setup] INFO: 4870 atoms, 597 residues read.\n[lightdock3_setup] INFO: Calculating reference points for receptor /home/justin/InstaDeep/mutation_test/6m0j_Spike_output.pdb...\n[lightdock3_setup] INFO: Reference points for receptor found, skipping\n[lightdock3_setup] INFO: Done.\n[lightdock3_setup] INFO: Calculating reference points for ligand /home/justin/InstaDeep/mutation_test/6m0j_ACE2_output.pdb...\n[lightdock3

In [ ]:
# Perform docking bwtween the orginal protiens: Spike protein (Prot_A) and ACE2 (Prot_B)
perform_docking(pathlib.Path('6m0j_Spike_output.pdb'), pathlib.Path('7z0x_antibody_output.pdb'), 50, 1, working_path=pathlib.Path('/home/justin/InstaDeep/Spike_antibody'))

(b'[lightdock3_setup] INFO: Ignoring OXT atoms\n[lightdock3_setup] INFO: Ignoring Hydrogen atoms\n[lightdock3_setup] INFO: Ignoring water\n[lightdock3_setup] INFO: Reading structure from 6m0j_Spike_output.pdb PDB file...\n[lightdock3_setup] INFO: 1536 atoms, 194 residues read.\n[lightdock3_setup] INFO: Ignoring OXT atoms\n[lightdock3_setup] INFO: Ignoring Hydrogen atoms\n[lightdock3_setup] INFO: Ignoring water\n[lightdock3_setup] INFO: Reading structure from 7z0x_antibody_output.pdb PDB file...\n[lightdock3_setup] INFO: 3251 atoms, 439 residues read.\n[lightdock3_setup] INFO: Calculating reference points for receptor 6m0j_Spike_output.pdb...\n[lightdock3_setup] INFO: Reference points for receptor found, skipping\n[lightdock3_setup] INFO: Done.\n[lightdock3_setup] INFO: Calculating reference points for ligand 7z0x_antibody_output.pdb...\n[lightdock3_setup] INFO: Reference points for ligand found, skipping\n[lightdock3_setup] INFO: Done.\n[lightdock3_setup] INFO: Saving processed structu

### Identify contributing residues

In [7]:
# Analyse the docking result of the original proteins (Spike protein (Prot_A) and ACE2 (Prot_C)) and generate the binding structure with the best score
docking_result_AC = analysize_docking_result(receptor_pdb=pathlib.Path('6m0j_Spike_output.pdb'), ligand_pdb=pathlib.Path('6m0j_ACE2_output.pdb'), step=50, generate_best_pdb=True, pdb_num_for_generated=200, working_path=working_dir)
print (f"Index: {docking_result_AC[0]}, Highest docking score: {docking_result_AC[1]}, Best PDB path:{docking_result_AC[2]}")

(b'[generate_conformations] INFO: Reading lightdock_6m0j_Spike_output.pdb receptor PDB file...\n[generate_conformations] INFO: 1536 atoms, 194 residues read.\n[generate_conformations] INFO: Reading lightdock_6m0j_ACE2_output.pdb ligand PDB file...\n[generate_conformations] INFO: 4870 atoms, 597 residues read.\n[generate_conformations] INFO: Read 200 coordinate lines\n[generate_conformations] INFO: Generated 200 conformations\n', None)
Index: 196, Highest docking score: 225.9345, Best PDB path:/home/justin/InstaDeep/mutation_test/protein_design_trial/Trial_4/docking_AB/swarm_3/lightdock_196.pdb


In [8]:
# Identify the contributing residues to the binding (Spike protein (Prot_A) and ACE2 (Prot_C)) based on the binding strucre from above. Here use 2.5 Å as the distance cutoff
contact_residue_dict_AC = identify_contributing_residues(pathlib.Path("/home/justin/InstaDeep/mutation_test/swarm_9/lightdock_149.pdb"), 2.5)
print (contact_residue_dict_AC)
print (len(contact_residue_dict_AC["contact_residue_dict"]))

 PyMOL not running, entering library mode (experimental)
{'contact_residue_dict': {'333': ('THR', 'E'), '334': ('ASN', 'E'), '335': ('LEU', 'E'), '336': ('CYS', 'E'), '351': ('TYR', 'E'), '352': ('ALA', 'E'), '353': ('TRP', 'E'), '354': ('ASN', 'E'), '355': ('ARG', 'E'), '356': ('LYS', 'E'), '357': ('ARG', 'E'), '360': ('ASN', 'E'), '361': ('CYS', 'E'), '362': ('VAL', 'E'), '363': ('ALA', 'E'), '364': ('ASP', 'E'), '365': ('TYR', 'E'), '366': ('SER', 'E'), '388': ('ASN', 'E'), '389': ('ASP', 'E'), '390': ('LEU', 'E'), '391': ('CYS', 'E'), '392': ('PHE', 'E'), '396': ('TYR', 'E'), '397': ('ALA', 'E'), '398': ('ASP', 'E'), '411': ('ALA', 'E'), '412': ('PRO', 'E'), '413': ('GLY', 'E'), '414': ('GLN', 'E'), '419': ('ALA', 'E'), '422': ('ASN', 'E'), '423': ('TYR', 'E'), '424': ('LYS', 'E'), '425': ('LEU', 'E'), '426': ('PRO', 'E'), '427': ('ASP', 'E'), '428': ('ASP', 'E'), '429': ('PHE', 'E'), '430': ('THR', 'E'), '457': ('ARG', 'E'), '458': ('LYS', 'E'), '459': ('SER', 'E'), '460': ('ASN',

In [11]:
# Analyse the docking result of the orginal protiens (Spike protein (Prot_A) and antibody (Prot_B)) and generate the binding strucure with the best score
docking_result_AB = analysize_docking_result(receptor_pdb=pathlib.Path('6m0j_Spike_output.pdb'), ligand_pdb=pathlib.Path('7z0x_antibody_output.pdb'), step=50, generate_best_pdb=True, pdb_num_for_generated=200, working_path=pathlib.Path('/home/justin/InstaDeep/Spike_antibody'))
print (f"Index: {docking_result_AB[0]}, Highest docking score: {docking_result_AB[1]}, Best PDB path:{docking_result_AB[2]}")

(b'[generate_conformations] INFO: Reading lightdock_6m0j_Spike_output.pdb receptor PDB file...\n[generate_conformations] INFO: 1536 atoms, 194 residues read.\n[generate_conformations] INFO: Reading lightdock_7z0x_antibody_output.pdb ligand PDB file...\n[generate_conformations] INFO: 3251 atoms, 439 residues read.\n[generate_conformations] INFO: Read 200 coordinate lines\n[generate_conformations] INFO: Generated 200 conformations\n', None)
Index: 59, Highest docking score: 180.8483, Best PDB path:/home/justin/InstaDeep/Spike_antibody/swarm_2/lightdock_59.pdb


In [12]:
# Identify the contributing residues to the binding (Spike protein (Prot_A) and antibody (Prot_B)) based on the binding strucre from above. Here use 2.5 Å as the distance cutoff
contact_residue_dict_AB = identify_contributing_residues(pathlib.Path("/home/justin/InstaDeep/Spike_antibody/swarm_2/lightdock_59.pdb"), 2.5)
print (contact_residue_dict_AB)
print (len(contact_residue_dict_AB["contact_residue_dict"]))

{'contact_residue_dict': {'333': ('THR', 'E'), '334': ('ASN', 'E'), '335': ('LEU', 'E'), '336': ('CYS', 'E'), '351': ('TYR', 'E'), '352': ('ALA', 'E'), '353': ('TRP', 'E'), '354': ('ASN', 'E'), '355': ('ARG', 'E'), '356': ('LYS', 'E'), '357': ('ARG', 'E'), '360': ('ASN', 'E'), '361': ('CYS', 'E'), '362': ('VAL', 'E'), '363': ('ALA', 'E'), '364': ('ASP', 'E'), '365': ('TYR', 'E'), '366': ('SER', 'E'), '388': ('ASN', 'E'), '389': ('ASP', 'E'), '390': ('LEU', 'E'), '391': ('CYS', 'E'), '392': ('PHE', 'E'), '396': ('TYR', 'E'), '397': ('ALA', 'E'), '398': ('ASP', 'E'), '411': ('ALA', 'E'), '412': ('PRO', 'E'), '413': ('GLY', 'E'), '414': ('GLN', 'E'), '419': ('ALA', 'E'), '422': ('ASN', 'E'), '423': ('TYR', 'E'), '424': ('LYS', 'E'), '425': ('LEU', 'E'), '426': ('PRO', 'E'), '427': ('ASP', 'E'), '428': ('ASP', 'E'), '429': ('PHE', 'E'), '430': ('THR', 'E'), '457': ('ARG', 'E'), '458': ('LYS', 'E'), '459': ('SER', 'E'), '460': ('ASN', 'E'), '461': ('LEU', 'E'), '462': ('LYS', 'E'), '463': (

### Mutate residues of Spike protein (Prot_A), dock new Spike protein with mutated residues and rescore the docking 

In [21]:
# Perform scanning combination of different mutant residues to find the parameter combination with a higher score  
orig_score_AB = docking_result_AB[1]
orig_score_AC = docking_result_AC[1]
print (orig_score_AB, orig_score_AC)

orig_A_pdb_path = pathlib.Path(working_dir, "6m0j_Spike_output.pdb")
orig_ligand_B_path = pathlib.Path(working_dir, "7z0x_antibody_output.pdb")
orig_ligand_C_path = pathlib.Path(working_dir, "6m0j_ACE2_output.pdb")

shared_keys_dict = contact_residue_dict_AB["contact_residue_dict"].keys() & contact_residue_dict_AC["contact_residue_dict"].keys()

contact_residue_dict={}
contact_residue_dict_temp = {key:contact_residue_dict_AB["contact_residue_dict"][key] for key in shared_keys_dict if key in contact_residue_dict_AB["contact_residue_dict"]}
contact_residue_dict["contact_residue_dict"] = dict(list(contact_residue_dict_temp.items())[:7])
print (contact_residue_dict)
print (len(contact_residue_dict_temp))
study = optuna.create_study()
study.optimize(lambda trial:objective(trial, orig_score_AB, orig_score_AC, orig_A_pdb_path, orig_ligand_B_path, orig_ligand_C_path, contact_residue_dict), n_trials=6)
study.get_trials
study.best_params

[I 2023-03-20 00:44:51,140] A new study created in memory with name: no-name-8405b401-ed69-4492-819c-6af4f55b1f23


180.8483 225.9345
{'contact_residue_dict': {'428': ('ASP', 'E'), '468': ('ILE', 'E'), '333': ('THR', 'E'), '520': ('ALA', 'E'), '411': ('ALA', 'E'), '424': ('LYS', 'E'), '430': ('THR', 'E')}}
64
Trial:0: ('D', 'E', 'D', 'P', 'I', 'I', 'I')
6m0j_Spike_output
Total 7 residues to be mutated
PyMOL>refresh_wizard
Selected!
 ExecutiveRMSPairs: RMSD =    0.025 (4 to 4 atoms)
 Mutagenesis: 9 rotamers loaded.
 Rotamer 4/9, strain=22.63
1: Mutating from E/428/ to ASP
Selected!
 ExecutiveRMSPairs: RMSD =    0.020 (4 to 4 atoms)
 Mutagenesis: 12 rotamers loaded.
 Rotamer 8/12, strain=33.61
2: Mutating from E/468/ to GLU
Selected!
 ExecutiveRMSPairs: RMSD =    0.030 (4 to 4 atoms)
 Mutagenesis: no phi/psi, using backbone-independent rotamers.
 Mutagenesis: 9 rotamers loaded.
 Rotamer 2/9, strain=36.78
3: Mutating from E/333/ to ASP
Selected!
 ExecutiveRMSPairs: RMSD =    0.058 (4 to 4 atoms)
 Mutagenesis: 2 rotamers loaded.
 Rotamer 1/2, strain=35.70
4: Mutating from E/520/ to PRO
Selected!
 Execut

[I 2023-03-20 01:09:12,957] Trial 0 finished with value: 61688.96250000001 and parameters: {'mutpoint1': 'D', 'mutpoint2': 'E', 'mutpoint3': 'D', 'mutpoint4': 'P', 'mutpoint5': 'I', 'mutpoint6': 'I', 'mutpoint7': 'I'}. Best is trial 0 with value: 61688.96250000001.


(111, 164.257)
trail:0 score: (192.3108, 164.257)
Trial:1: ('K', 'R', 'H', 'W', 'K', 'K', 'D')
6m0j_Spike_output
PyMOL>refresh_wizard
Total 7 residues to be mutated
Selected!
 ExecutiveRMSPairs: RMSD =    0.022 (4 to 4 atoms)
 Mutagenesis: 17 rotamers loaded.
 Rotamer 1/17, strain=25.00
1: Mutating from E/428/ to LYS
Selected!
 ExecutiveRMSPairs: RMSD =    0.024 (4 to 4 atoms)
 Mutagenesis: 20 rotamers loaded.
 Rotamer 16/20, strain=30.86
2: Mutating from E/468/ to ARG
Selected!
 ExecutiveRMSPairs: RMSD =    0.043 (4 to 4 atoms)
 Mutagenesis: no phi/psi, using backbone-independent rotamers.
 Mutagenesis: 9 rotamers loaded.
 Rotamer 1/9, strain=33.39
3: Mutating from E/333/ to HIS
Selected!
 ExecutiveRMSPairs: RMSD =    0.013 (4 to 4 atoms)
 Mutagenesis: 5 rotamers loaded.
 Rotamer 1/5, strain=30.96
4: Mutating from E/520/ to TRP
Selected!
 ExecutiveRMSPairs: RMSD =    0.026 (4 to 4 atoms)
 Mutagenesis: 14 rotamers loaded.
 Rotamer 9/14, strain=25.86
5: Mutating from E/411/ to LYS
Selec

[I 2023-03-20 01:33:36,198] Trial 1 finished with value: 26775.933200000014 and parameters: {'mutpoint1': 'K', 'mutpoint2': 'R', 'mutpoint3': 'H', 'mutpoint4': 'W', 'mutpoint5': 'K', 'mutpoint6': 'K', 'mutpoint7': 'D'}. Best is trial 1 with value: 26775.933200000014.


...

[I 2023-03-20 02:46:56,057] Trial 4 finished with value: 67822.79670000002 and parameters: {'mutpoint1': 'K', 'mutpoint2': 'F', 'mutpoint3': 'D', 'mutpoint4': 'W', 'mutpoint5': 'D', 'mutpoint6': 'D', 'mutpoint7': 'K'}. Best is trial 1 with value: 26775.933200000014.


(175, 158.1404)
trail:4 score: (209.545, 158.1404)
Trial:5: ('L', 'R', 'N', 'E', 'D', 'I', 'S')
6m0j_Spike_output
Total 7 residues to be mutated
PyMOL>refresh_wizard
Selected!
 ExecutiveRMSPairs: RMSD =    0.025 (4 to 4 atoms)
 Mutagenesis: 3 rotamers loaded.
 Rotamer 1/3, strain=29.38
1: Mutating from E/428/ to LEU
Selected!
 ExecutiveRMSPairs: RMSD =    0.024 (4 to 4 atoms)
 Mutagenesis: 20 rotamers loaded.
 Rotamer 16/20, strain=30.86
2: Mutating from E/468/ to ARG
Selected!
 ExecutiveRMSPairs: RMSD =    0.030 (4 to 4 atoms)
 Mutagenesis: no phi/psi, using backbone-independent rotamers.
 Mutagenesis: 18 rotamers loaded.
 Rotamer 1/18, strain=38.51
3: Mutating from E/333/ to ASN
Selected!
 ExecutiveRMSPairs: RMSD =    0.013 (4 to 4 atoms)
 Mutagenesis: 17 rotamers loaded.
 Rotamer 2/17, strain=30.86
4: Mutating from E/520/ to GLU
Selected!
 ExecutiveRMSPairs: RMSD =    0.025 (4 to 4 atoms)
 Mutagenesis: 7 rotamers loaded.
 Rotamer 6/7, strain=32.12
5: Mutating from E/411/ to ASP
Sele

[I 2023-03-20 03:11:26,479] Trial 5 finished with value: 89540.93850000002 and parameters: {'mutpoint1': 'L', 'mutpoint2': 'R', 'mutpoint3': 'N', 'mutpoint4': 'E', 'mutpoint5': 'D', 'mutpoint6': 'I', 'mutpoint7': 'S'}. Best is trial 1 with value: 26775.933200000014.


(63, 136.3884)
trail:5 score: (175.6868, 136.3884)


{'mutpoint1': 'K',
 'mutpoint2': 'R',
 'mutpoint3': 'H',
 'mutpoint4': 'W',
 'mutpoint5': 'K',
 'mutpoint6': 'K',
 'mutpoint7': 'D'}

In [22]:
# List the detailed scores of each step
for step in range(len(study.trials)):
    print(f"{step}:", study.trials[step].user_attrs)

0: {'score_AB': 192.3108, 'score_AC': 164.257, 'parameters': ('D', 'E', 'D', 'P', 'I', 'I', 'I'), 'loss': 61688.96250000001}
1: {'score_AB': 193.4815, 'score_AC': 199.1712, 'parameters': ('K', 'R', 'H', 'W', 'K', 'K', 'D'), 'loss': 26775.933200000014}
2: {'score_AB': 200.3793, 'score_AC': 176.4455, 'parameters': ('K', 'N', 'N', 'P', 'I', 'K', 'I'), 'loss': 49508.53100000001}
3: {'score_AB': 186.2372, 'score_AC': 176.6076, 'parameters': ('K', 'F', 'D', 'W', 'K', 'K', 'I'), 'loss': 49332.28890000002}
4: {'score_AB': 209.545, 'score_AC': 158.1404, 'parameters': ('K', 'F', 'D', 'W', 'D', 'D', 'K'), 'loss': 67822.79670000002}
5: {'score_AB': 175.6868, 'score_AC': 136.3884, 'parameters': ('L', 'R', 'N', 'E', 'D', 'I', 'S'), 'loss': 89540.93850000002}
